### Задание 1



#### 1a. Найти $\sqrt[k]{a}$ помощью метода бисекции.

In [2]:
def bin_search(f, l, r):
  while r - l > 1e-15:
    m = (l + r) / 2
    if f(m) > 0:
      r = m
    else:
      l = m
  return l

def root1(k: float, a: float):
  assert a > 0
  assert k > 0

  l, r = 0, 100
  while r ** k < a:
    r *= 100
  
  return bin_search(lambda x : x ** k - a, l, r)

print(root1(2, 4))
print(root1(3, 27))
print(root1(1, 7))

1.9999999999999998
3.0
7.0




#### 1b. Найти $\sqrt[k]{a}$ помощью метода Ньютона.

$f(x) = x^k - a$

$x_{n + 1} = x_n + \frac{x_n^k - a}{k \cdot x_n^{k-1}}$



In [11]:
def root2(k: float, a: float, iterations=1000):
    x = a
    for _ in range(iterations):
        x -= (x**k - a) / (k * x**(k - 1))

    return x

print(root2(2, 4))
print(root2(3, 27))
print(root2(1, 7))

2.0
3.0
7.0


### Задание 2

#### 2b. Дан многочлен P степени не больше 5 и отрезок [L; R]. 

Для каждого $[L_i; R_i]$ найти соответствующий корень.

In [4]:
def find_roots_by_segments(a, b, c, d, e, f, segments):
  """Finds a*x^5 + b*x^4 + c*x^3 + d*x^2 + e*x + f roots using binary search"""
  def func(x):
    return a * x**5 + b * x**4 + c * x**3 + d * x**2 + e * x + f

  roots = []
  for (l, r) in segments:
    while r - l > 1e-15:
      m = (l + r) / 2
      if func(m) * func(l) <= 0:
        r = m
      else:
        l = m
    roots.append(l)
  return roots

# roots: -1, 0, 1
print(find_roots_by_segments(0, 0, 1, 0, -1, 0, [(-1.5, -0.5), (-0.3, 0.3), (0.5, 1.5)]))

[-1.0000000000000009, -5.329070518200751e-16, 0.9999999999999991]


#### 2a. Дан многочлен P степени не больше 5 и отрезок [L; R]. 
Локализовать корни P, т.е. найти непересекающиеся отрезки $[L_1; R_1], . . . , [L_m; R_m]$
такие, что $P(L_i)P(R_i) < 0$.

In [5]:
def find_segments(a, b, c, d, e, f, L, R):
  """Returns list of segments with localized roots for
   a*x^5 + b*x^4 + c*x^3 + d*x^2 + e*x + f on [L, R)"""

  def func(x):
    return a * x**5 + b * x**4 + c * x**3 + d * x**2 + e * x + f

  if a == 0 and b == 0 and c == 0 and d == 0:
      # Function e * x + f
      if (e * L + f) * (e * R + f) < 0 or e * L + f == 0:
        return [(L, R)]
      else:
        return []

  # Find derivative roots.
  der_segments = find_segments(0, 5 * a, 4 * b, 3 * c, 2 * d, e, L, R)
  der_roots = find_roots_by_segments(0, 5 * a, 4 * b, 3 * c, 2 * d, e, der_segments)

  der_roots.insert(0, L)
  der_roots.append(R)
  # All segments between derivative roots.  
  some_segments = list(zip(der_roots, der_roots[1:]))

  # Return only segments with root between.
  eps = 1e-15
  segments = []
  for (l, r) in some_segments:
    if func(l) * func(r) < 0 and abs(func(l)) > eps and abs(func(r)) > eps:
      segments.append((l, r))
    if abs(func(l)) < eps:
      segments.append((l, l))
  return segments

print(find_segments(1, 0, -5, 0, 4, 0, -10, 10))  # roots: -2, -1, 0, 1, 2
print()



def find_roots(a, b, c, d, e, f, L, R):
  """Returns list of all a * x** + b * x**4 + c * x**3 + d * x**2 + e * x  + f in [L, R)"""
  return find_roots_by_segments(a, b, c, d, e, f, find_segments(a, b, c, d, e, f, L, R))

print(find_roots(1, 0, -5, 0, 4, 0, -10, 10))    # roots -2, -1, 0, 1, 2
print(find_roots(1, 0, -5, 0, 4, 0, -1.1, 1.1))  # roots -1, 0, 1
print(find_roots(0, 1, 0, 0, 0, 0, -10, 10))     # roots 0

[(-10, -1.6444328681582694), (-1.6444328681582694, -0.5439122559023383), (-0.5439122559023383, 0.5439122559023378), (0.5439122559023378, 1.644432868158268), (1.644432868158268, 10)]

[-2.000000000000001, -1.0000000000000002, -2.220446049250313e-16, 0.9999999999999998, 1.9999999999999998]
[-1.0000000000000009, -6.106226635438361e-16, 0.999999999999999]
[-5.551115123125783e-16]


#### 2c. Дан многочлен P степени не больше 5 и отрезок [L; R].

Найти глобальный минимум P на отрезке $[L; R]$.

In [6]:
def find_global_minimum(a, b, c, d, e, f, L, R):
  """Returns global minimum for a * x**5 + b * x**4 + c * x**3 + d * x**2 + e * x + f"""

  def func(x):
    return a * x**5 + b * x**4 + c * x**3 + d * x**2 + e * x + f
  
  # Find derivative roots.
  roots = find_roots(0, 5 * a, 4 * b, 3 * c, 2 * d, e, L, R)

  return min([func(root) for root in roots + [L, R]])

print(find_global_minimum(1, -1, -5, -3, 1, 1, -2, 4))  # minimum -37.029
print(find_global_minimum(1, 0, 0, 0, 0, 0, -2, 4))     # minimum -32
print(find_global_minimum(0, 1, 0, 0, 0, 0, -2, 4))     # minimum 0

-37.02948688415573
-32
3.8893845486632136e-62


### Задание 3

#### 3a. При a, b, c > 0 найти минимум функции $e^{ax} + e^{−bx} + c(x − d)^2$ с использованием метода бисекции.

$(e^{ax} + e^{−bx} + c(x − d)^2)' = ae^{ax} - be^{-bx} + 2c(x - d)$ - докажем, что есть ровно один корень.

В $x \rightarrow +\infty$ функция $+\infty$, в $x \rightarrow -\infty$ функция $-\infty$. 

$(ae^{ax} - be^{-bx} + 2c(x - d))' = a^2e^{ax} + b^2e^{-bx} + 2c > 0$, а значит у $ae^{ax} - be^{-bx} + 2c(x - d)$ ровно один корень. Значит изначальная функция $e^{ax} + e^{−bx} + c(x − d)^2$ сначала убывает, затем возрастает; ее минимум --- в корне функции $ae^{ax} - be^{-bx} + 2c(x - d)$. Тогда можно применить метод бисекции к $ae^{ax} - be^{-bx} + 2c(x - d)$.

In [9]:
import math

def find_root1(a, b, c, d):
  assert a > 0 and b > 0 and c > 0

  def f(x):
    return a * math.e ** (a * x) - b * math.e ** (-b * x) + 2 * c * (x - d)
  
  l, r = -2, 2
  while f(l) > 0:
    l *= 2
  while f(r) < 0:
    r *= 2

  x = bin_search(f, l, r)
  return x, math.e ** (a * x) + math.e ** (-b * x) + c * (x - d) ** 2

print(find_root1(1, 1, 1, 1))  # minimum - 2.505 in x = 0.49

(0.4900730684805472, 2.505042602807782)


#### 3b. При a, b, c > 0 найти минимум функции $e^{ax} + e^{−bx} + c(x − d)^2$ с использованием метода Ньютона.

Найдем ноль производной исходной функции: $ae^{ax} - be^{-bx} + 2c(x - d) = f(x)$

$x_{n+1} = x_n - \frac{ae^{ax} - be^{-bx} + 2c(x - d)}{a^2e^{ax} + b^2e^{-bx} + 2c}$

In [8]:
def find_root2(a, b, c, d, iteration_n=1000):
    x = 0
    for _ in range(iteration_n):
        x -= (a * math.e**(a * x) - b * math.e**(-b * x) + 2 * c * (x - d)) /\
             (a**2 * math.e**(a * x) + b**2 * math.e**(-b * x) + 2 * c)
    return x, math.e ** (a * x) + math.e ** (-b * x) + c * (x - d) ** 2

print(find_root2(1, 1, 1, 1))  # minimum - 2.505 in x = 0.49

(0.4900730684805477, 2.505042602807782)


#### 3c. При a, b, c > 0 найти минимум функции $e^{ax} + e^{−bx} + c(x − d)^2$ с использованием тернарного поиска.

В задании 3a мы поняли, что $e^{ax} + e^{−bx} + c(x − d)^2$ сначала возрастает, а потом убывает. Тогда можно использовать тернарный поиск.

In [ ]:
def find_root3(a, b, c, d):
  assert a > 0 and b > 0 and c > 0

  def f_der(x):
    return a * math.e ** (a * x) - b * math.e ** (-b * x) + 2 * c * (x - d)
  
  def f(x):
    return math.e ** (a * x) + math.e ** (-b * x) + c * (x - d) ** 2
  
  l, r = -2, 2
  while f_der(l) > 0:
    l *= 2
  while f_der(r) < 0:
    r *= 2

  while r - l > 1e-15:
    m1 = (2 * l + r) / 3
    m2 = (l + 2 * r) / 3
    if f(m1) < f(m2):
      r = m2
    else:
      l = m1
  return l, f(l)

print(find_root3(1, 1, 1, 1))

(0.4900730781563903, 2.5050426028077823)
